In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 11.1 MB/s eta 0:00:00


**Importing Necessary Libraries**

In [ ]:
import pandas as pd
import numpy as np
import re
import nlpaug.augmenter.word as naw
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

**Loading the Dataset**

In [ ]:
#reading the dataset in csv format

data = pd.read_csv('/content/drive/MyDrive/Datasets/gittercom_annotated_data.csv')

**Text Preprocessing**


In [ ]:
def text_preprocessing(text):
    #removing punctuation from text data
    text = re.sub(r'[^\w\s]', '', text)

    #removing special symbols from text data
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    #replacing usernames in text data with '<username>'
    text = re.sub(r'@[^\s]+', '<username>', text)

    #replacing url in text data with '<url>' 
    text = re.sub(r'http\S+|www\S+|\S+\.com\S+', '<url>', text)
    
    #replacing code snippets in text data with '<code>' 
    text = re.sub(r'`[^`]+`', '<code>', text)
    
    return text

**Applying Text Preprocessing to the Text Data**

In [ ]:
#applying preprocessing techniques to the text data ('message' column holds all textual communications,i.e., text data)

data['message'] = data['message'].apply(text_preprocessing)

**Counting the Total Number of Instances of Our Dataset**

In [ ]:
data.shape[0]

1000

In [ ]:
#exploring first five rows of our dataset

data.head()

,Channel,message,category
0,Cucumber,Hi Team I just recently upgraded our cucumberj...,fear
1,Cucumber,So github is trying to replace irc P,fear
2,Cucumber,aslakhellesoy Thanks seems like I was using o...,joy
3,Cucumber,Sidkiyassine just call the methods directly,surprise
4,Cucumber,Hello guys need a helpI want to call the run m...,fear


In [ ]:
#exploring the instances of message column that holds text data

data['message']

0      Hi Team I just recently upgraded our cucumberj...
1                   So github is trying to replace irc P
2      aslakhellesoy Thanks  seems like I was using o...
3            Sidkiyassine just call the methods directly
4      Hello guys need a helpI want to call the run m...
                             ...                        
995    Besides the unzip issue which I saw you solved...
996    Hopefully now that the unzip issue is fixed we...
997     I can see that being a problem with my community
998                       which would slow down installs
999         Thanks yet again Ill try your suggestion now
Name: message, Length: 1000, dtype: object

**Splitting the Dataset for Classification**

In [ ]:
X = data['message'].values
y = data['category'].values

#splitting the data into training and testing sets (70% for training and 30% for testing)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Data Augmentation of training data using Synonym**

In [ ]:
augmented_data = []
augmented_category = []
no_of_targeted_augmentation = 30000

#initializing SynonymAug class of nlp aug library that leverage semantic meaning to substitute word using synonym
data_augmentation = naw.SynonymAug(aug_src='wordnet')

while len(augmented_data) < no_of_targeted_augmentation:
    for message, category in zip(X_train, y_train):
        augmented_text = data_augmentation.augment(message)
        augmented_data.append(augmented_text)
        augmented_category.append(category)
        if len(augmented_data) == no_of_targeted_augmentation:
            break

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
X_train = X_train.tolist() + augmented_data[:no_of_targeted_augmentation]
y_train = y_train.tolist() + augmented_category[:no_of_targeted_augmentation]

**Counting the Total Number of Instances of training data After Data Augmentation**

In [ ]:
len(X_train)

30700

**Dropping Any Possible Duplicate Instance(s) of Text Data Due to Performing Data Augmentation**

In [ ]:
data.drop_duplicates(subset='message', inplace=True)

In [ ]:
len(X_train)

30700

**Exploring Augmented Data**

In [ ]:
for message, category in zip(X_train, y_train):
    print("message:", message)
    print("category:", category)

Streaming output truncated to the last 5000 lines.
message: ['also jdubois cannot find the pourboire folder <url >']
category: sadness
message: ['hawaii i am running into cucumberspring124 v springboot130 issues <url> javalangArrayStoreException sunreflectannotationTypeNotPresentExceptionProxy']
category: fear
message: ['ane care i could do this sort of oeuvre for free as well only unfortunately its non possible right nowadays but it would be a dream come rightful to work total time on a projection like this']
category: sadness
message: ['So 1 would merely prolong that a litle bit possibly with a few notes to that effect and everything would exist all good']
category: joy
message: ['Tips and legerdemain look awesome']
category: joy
message: ['Ping pink ping Iu2019d like to perplex the jspm workflow for notorious publish tonight D Just 380 needs a fix or be information technology my fault']
category: joy
message: ['Intelligence community an finish this with this']
category: joy
message:

**Converting Augmented Data Back into Strings for Feature Extraction**

In [ ]:
X_train = [' '.join(x) for x in X_train]

**Preparing the Data Before Feeding It to the BiLSTM Model**

In [ ]:
#converting text data to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_length = 100  

#padding the sequences
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')


**Performing One-Hot Encoding**

In [ ]:
one_hot_encoder = OneHotEncoder()
y_train = one_hot_encoder.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_test = one_hot_encoder.transform(np.array(y_test).reshape(-1, 1)).toarray()


**Building BiLSTM Model**

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100  

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          692300    
                                                                 
 bidirectional (Bidirectiona  (None, 100, 128)         84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Training the Model**

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)

Epoch 1/50
480/480 [==============================] - 64s 103ms/step - loss: 0.5739 - accuracy: 0.7842 - val_loss: 3.0581 - val_accuracy: 0.3967
Epoch 2/50
480/480 [==============================] - 15s 31ms/step - loss: 0.0762 - accuracy: 0.9754 - val_loss: 3.9729 - val_accuracy: 0.4467
Epoch 3/50
480/480 [==============================] - 14s 29ms/step - loss: 0.0504 - accuracy: 0.9817 - val_loss: 4.7263 - val_accuracy: 0.4067
Epoch 4/50
480/480 [==============================] - 12s 25ms/step - loss: 0.0525 - accuracy: 0.9816 - val_loss: 4.2089 - val_accuracy: 0.4133
Epoch 5/50
480/480 [==============================] - 12s 25ms/step - loss: 0.0477 - accuracy: 0.9823 - val_loss: 4.4698 - val_accuracy: 0.4433
Epoch 6/50
480/480 [==============================] - 13s 28ms/step - loss: 0.0439 - accuracy: 0.9837 - val_loss: 4.9763 - val_accuracy: 0.4300
Epoch 7/50
480/480 [==============================] - 11s 22ms/step - loss: 0.0402 - accuracy: 0.9844 - val_loss: 5.3660 - val_accuracy

**Testing and Evaluating the Model**

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

#Generating Classification Report of the Model for Computing Precision, Recall and F1-Score for Every Six Emotion Categories
clsf_rprt = classification_report(np.argmax(y_test, axis=1), y_pred_labels)
print("Classification_report:\n", clsf_rprt)

10/10 [==============================] - 1s 12ms/step
Classification_report:
               precision    recall  f1-score   support

           0       0.19      0.11      0.14        28
           1       0.38      0.38      0.38        78
           2       0.53      0.66      0.59        88
           3       0.25      0.33      0.29         9
           4       0.39      0.38      0.38        88
           5       0.00      0.00      0.00         9

    accuracy                           0.42       300
   macro avg       0.29      0.31      0.30       300
weighted avg       0.39      0.42      0.41       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#calculating precision score
precision = precision_score(np.argmax(y_test, axis=1), y_pred_labels, average='weighted')

#calculating recall score
recall = recall_score(np.argmax(y_test, axis=1), y_pred_labels, average='weighted')

#calculating f1-score score
f1 = f1_score(np.argmax(y_test, axis=1), y_pred_labels, average='weighted')

#calculating accuracy score
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_labels)

print("Precision (weighted):", precision)
print("Recall (weighted):", recall)
print("F1-Score (weighted):", f1)
print("Accuracy:", accuracy)

Precision (weighted): 0.39496797985249144
Recall (weighted): 0.42333333333333334
F1-Score (weighted): 0.40593041202706415
Accuracy: 0.42333333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
